<h3> area_alements </h3>

In [1]:
def area_elements(x_length, y_length, element_x_length, elements_bycolums_ratio):
    """
        Funkcja obliczająca ilość elementów dla kolejnych obszarów.
        input: x_length - np.array(), wektor długości kolejnego obszaru;
               y_length - np.array(), wektor szerokości kolejnego obszaru;
               element_x_length - int, szerokosc elementu;
               elements_bycolums_ratio - int, wysokosc elementu;
        output: elements_count - np.array(), tablica z ilością elementów dla kolejnych obszarów
    """

    elements_count = np.array([])  # tablica przechowująca liczbę elementów obszarów

    # Liczymy ilosc elementów dla kolejnych obszarów

    for i in range(len(x_length)):
        elements_count = np.append(elements_count, 2 * (
                    x_length[i] * y_length[i] / element_x_length / elements_bycolums_ratio))  # 2* bo trójkąty
    return elements_count


<h3> important_nodes </h3> 

In [2]:
def important_nodes(wall, obstacles):
    """
        funkcja wyliczająca wierzchołki potrzebne do generowania obszarów, wraz z ich długością i szerokością.
        input: wall = np.array([w1,w2,w3,w4]), gdzie w_i = lista, [x_y,y_i];
        obstacles = np.array([[o_1_1,o_1_2,o_1_3,o_1_4],...,[o_q_1,o_q_2,o_q_3,o_q_4]]), gdzie o_q_j = [x_q_j,y_q_j];
        output: important_node np.array((0, 2), int) tablica wierzchołków generujących;
        x_length np.array([]), y_length np.array([]) - wymiary kolejnych obszarów w 2 tablicach,
        reprezentujących szerokość i wysokość
    """

    # Inicjujemy tablice, które będziemy zwracać

    important_node = np.empty((0, 2), int)
    x_length = np.array([])
    y_length = np.array([])

    # Sprawdzamy czy podano chociaż 1 przeszkodę

    if len(obstacles) != 0:

        x_partition = np.array([])  # podziałka na osi x ( zaznaczamy miejsca gdzie są zrzutowane wierzchołki przeszkód na oś x )
        wall_down = wall[0][1]  # wysokość sciany dolnej
        wall_up = wall[2][1]  # wysokość sciany górnej

        # Dodajemy podziały związane z wallem

        x_partition = np.append(x_partition, wall[0][0])
        x_partition = np.append(x_partition, wall[1][0])

        # Wyliczamy podziały związane z przeszkodami ( rzutowanie węzłów przeszkód na oś x )

        for i in range(len(obstacles)):
            if obstacles[i][0][0] not in x_partition:
                x_partition = np.append(x_partition, obstacles[i][0][0])
            if obstacles[i][1][0] not in x_partition:
                x_partition = np.append(x_partition, obstacles[i][1][0])

        x_partition = np.sort(x_partition)  # sortujemy podział na osi x
        height = wall[2][1] - wall[0][1]  # liczymy wysokość walla

        # Atrybuty przeszkód, uproszczą nam myślenie o algorytmie i pisanie algorytmu

        column_number = []  # nadaje numer kolumny do ktorej nalezy przeszkoda
        if_from_bottom = []  # czy przeszkoda ciagnie sie od samego dolu
        if_to_heaven = []  # czy przeszkoda ciagnie sie do samej gory
        if_first = []  # czy przeszkoda jest pierwsza w kolumnie
        if_last = []  # czy przeszkoda jest ostatnia w kolumnie
        if_first_on_left_wall = False  # czy pierwsza przeszkoda zaczyna sie na lewej scianie
        if_last_on_right_wall = False  # czy ostatnia przeszkoda kończy się na prawej ścianie
        j = 0  # indeks numeru kolumny

        # Sprawdzamy powyższe atrybuty przeszkód

        if obstacles[0][0][0] == wall[0][0]:
            if_first_on_left_wall = True
        if obstacles[-1][1][0] == wall[1][0]:
            if_last_on_right_wall = True
        for i in range(len(obstacles)):
            if obstacles[i][0][1] == wall_down:
                if_from_bottom.append(True)
            else:
                if_from_bottom.append(False)

            if obstacles[i][2][1] == wall_up:
                if_to_heaven.append(True)
            else:
                if_to_heaven.append(False)

            if i == 0:
                column_number.append(0)
            else:
                if obstacles[i][0][0] == obstacles[i - 1][0][0]:
                    column_number.append(j)
                else:
                    j += 1
                    column_number.append(j)

            if i == 0:
                if_first.append(True)
            elif column_number[i - 1] == column_number[i]:
                if_first.append(False)
            else:
                if_first.append(True)
            if i == len(obstacles) - 1:
                if_last.append(True)
            elif obstacles[i + 1][0][0] != obstacles[i][0][0]:
                if_last.append(True)
            else:
                if_last.append(False)
                
        # Pętla licząca tablice, które zwraca funkcja

        k = 0  # zmienna interacji podziału

        for i in range(len(obstacles)):
            if i != 0:  # jeśli przeszkoda nie jest pierwsza
                if obstacles[i - 1][1][0] != obstacles[i][0][0] and column_number[i] != column_number[i-1]:  # jeśli była przerwa pomiędzy kolumnami
                    if if_first[i]:  # przeszkoda pierwsza w kolumnie
                        # to dodajemy obszar bezprzeszkodowy
                        k += 1
                        important_node = np.concatenate((important_node, [[x_partition[0 + k], wall_down]]), axis=0)
                        x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                        y_length = np.append(y_length, height)
                        k += 1
                        # rozważamy co wnosi przeszkoda
                        if if_from_bottom[i]:
                            if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                                important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, obstacles[i + 1][0][1] - obstacles[i][3][1])
                            else:
                                important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                        else:
                            if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                                important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]),
                                                                axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, obstacles[i][0][1] - wall_down)
                            else:
                                important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]),
                                                                axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, obstacles[i][0][1] - wall_down)

                                important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                    else:
                        if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                            important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - wall_down)
                        else:
                            important_node = np.concatenate((important_node, [obstacles[i - 1][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - obstacles[i - 1][3][1])

                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                else:
                    if if_first[i]:  # przeszkoda pierwsza w kolumnie
                        k += 1
                        # rozważamy co wnosi przeszkoda
                        if if_from_bottom[i]:
                            if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                                important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, obstacles[i + 1][0][1] - obstacles[i][3][1], axis=0)
                            else:
                                important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                        else:
                            if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                                important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]),
                                                                axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, obstacles[i][0][1] - wall_down)
                            else:
                                important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]),
                                                                axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, obstacles[i][0][1] - wall_down)

                                important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                                y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                    else:
                        if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                            important_node = np.concatenate((important_node, [obstacles[i-1][3]]), axis=0)
                            # important_node = np.concatenate((important_node, [[obstacles[i][0][0], obstacles[i][0][1]]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - obstacles[i-1][3][1])
                        else:
                            important_node = np.concatenate((important_node, [obstacles[i - 1][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - obstacles[i - 1][3][1])

                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, wall_up - obstacles[i][3][1])
            else:
                if not if_first_on_left_wall:  # jeśli była przerwa
                    #  dodajemy obszar bezprzeszkodowy
                    important_node = np.concatenate((important_node, [[x_partition[0 + k], wall_down]]), axis=0)
                    x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                    y_length = np.append(y_length, height)
                    k += 1
                    # rozważamy co wnosi przeszkoda
                    if if_from_bottom[i]:
                        if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i + 1][0][1] - obstacles[i][3][1], axis=0)
                        else:
                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                    else:
                        if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                            important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - wall_down)
                        else:
                            important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - wall_down)

                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                else:
                    # rozważamy co wnosi przeszkoda
                    if if_from_bottom[i]:
                        if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i + 1][0][1] - obstacles[i][3][1], axis=0)
                        else:
                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, wall_up - obstacles[i][3][1])
                    else:
                        if not if_last[i]:  # tzn ze musi byc jeszcze jakas przeszkoda w tej kol
                            important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - wall_down)
                        else:
                            important_node = np.concatenate((important_node, [[obstacles[i][0][0], wall_down]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, obstacles[i][0][1] - wall_down)

                            important_node = np.concatenate((important_node, [obstacles[i][3]]), axis=0)
                            x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                            y_length = np.append(y_length, wall_up - obstacles[i][3][1])

        if i == len(obstacles) - 1:  # sprawdzenie czy ostatnia przeszkoda leży na prawej ścianie, czy też jest przerwa
            if not if_last_on_right_wall:
                k += 1  # zwiększamy podziałke
                important_node = np.concatenate((important_node, [[obstacles[i][1][0], wall_down]]), axis=0)
                x_length = np.append(x_length, x_partition[1 + k] - x_partition[0 + k])
                y_length = np.append(y_length, height)
        

    else:
        important_node = np.concatenate((important_node, [wall[0]]), axis=0)
        x_length = np.append(x_length, wall[1][0] - wall[0][0])
        y_length = np.append(y_length, wall[3][1] - wall[0][1])
        
    return important_node, x_length, y_length


<h3> meshgenerator </h3>

In [ ]:
def meshgenerator(wall, obstacles, elements_byrows, elements_bycolumns, debug_mode=False, scale_axes = 1):
    """
        Funkcja generująca siatkę.
        input: węzły wall i węzły przeszkód: lewy_dolny, prawy_dolny, prawy_górny, lewy_górny
                wall = np.array([w1,w2,w3,w4]), gdzie w_i = lista, [x_y,y_i];
                obstacles = np.array([[o_1_1,o_1_2,o_1_3,o_1_4],...,[o_q_1,o_q_2,o_q_3,o_q_4]]),
                { WAŻNE } gdzie o_q_j = [x_q_j,y_q_j], przeszkody podane najpierw wg kolumn od lewej do prawej i później wg wierszy od dołu do góry;  
                elements_byrows - int, liczba elementów w wierszu;
                elements_bycolumns - int, liczba elementów w kolumnie;
                debug_mode - True/False,  czy zwrócić dodatkowe obiekty.
                scale_axes - float, stała skalująca współrzędne węzłów
        output: array_elements_keys - list(), macierz elementów z węzłami w postaci ponumerowania;
        list_of_nodes- np.array(), macierz z ponumerowanymi węzłami;
                 dodatkowo w trybie debugowania: array_elements list() macierz elementów z węzłami
                 w postaci wektorów;
                 area_elements - np.array([how_many_elements, 3, 2]), tablica z ilością elementów
                 dla kolejnych obszarów

        dodatkowe informacje: potrzebne do wyliczania elementów węzły zapisane są w tablicy primary_nodes
        (chcemy numerować węzły i elementy począwszy
            od lewego dolnego rogu każdego z obszarów i iść wierszami do góry,
            aż do wynumerowania wszystkich węzłów i elementów)
    """
    
    # Zmienne

    list_of_nodes = []  # lista ponumerowanych wektorów
    new_elementkeys = []  # lista do której wkładamy kolejne wierzchołki tworzące bieżący element
    # nazwijmy tę tablicę tablicą wierzchołków pierwotnych
    length = wall[1][0] - wall[0][0]  # długość obszaru
    height = wall[2][1] - wall[0][1]  # wysokość obszaru
    
    if debug_mode:
        array_elements = []  # tablica utworzonych elementów, węzły podane w postaci wektorów

    # Liczymy wymiary kolejnych obszarów 

    primary_nodes, x_length, y_length = important_nodes(wall, obstacles)
    # primary_nodes nazywamy wierzchołkami pierwotnymi
    
    # Wymiary pojedynczego elementu
    
    element_x_length = length / elements_byrows
    element_y_length = height / elements_bycolumns
    
    # Liczymy możliwe koordynaty węzłów

    x_coords = np.linspace(wall[0][0], wall[1][0], num=int(np.around(length/element_x_length,0))+1)
    y_coords = np.linspace(wall[0][1], wall[2][1], num=int(np.around(height/element_y_length,0))+1)
    
    Z = np.ones((len(y_coords), len(x_coords))) * (-1)  # macierz reprezentacji obszaru

    # Główna pętla - tworzenie wierzchołków i elementów
    
    area_elements_list = area_elements(x_length, y_length, element_x_length, element_y_length)

    # zgarniamy ile elementów zawierają poszczególne obszary
    how_many_elements = int(sum(area_elements_list))  # zgarniamy liczbę elementów siatki

    array_elements_keys = np.zeros([3, how_many_elements],
                                   dtype=int)  # podanie wymiarów dla connectivity matrix, 3 bo trójkąty,
    # lista utworzonych elementów, węzły podane w postaci ponumerowania

    if debug_mode:
        array_elements = np.empty([how_many_elements, 3, 2],
                                   dtype=object)  # podanie wymiarów dla connectivity matrix o wierzchołkach w postaci wektorów
        
    i = 0  # numeracja elementów
    j = 0  # zmienna sprawdzająca koniec wiersza wewnątrz obszaru
    k = 0  # zmienna sprawdzająca koniec obszaru
    p = 0  # zmienna numerująca elementy

    for l in range(len(primary_nodes)):
        j = 0
        k = 0
        while k * element_y_length < y_length[l]:  # czy koniec obszaru
            while j * element_x_length < x_length[l]:  # czy koniec wiersza
                for q in range(2):  # dwa trójkąty z jednego prostokąta
                    if q == 0:  # trójkąt parzysty
                        new_element = [
                            [((primary_nodes[l, 0] + j * element_x_length)), ((primary_nodes[l, 1] + k * element_y_length))],
                            [((primary_nodes[l, 0] + (j + 1) * element_x_length)), ((primary_nodes[l, 1] + k * element_y_length))],
                            [((primary_nodes[l, 0] + j * element_x_length)), ((primary_nodes[l, 1] + (k + 1) * element_y_length))]]
                        # tworzenie nowego elementu

                        for m in range(3):  # numeracja nowych wierzchołków
                            x = new_element[m][0]  # liczymy współrzędne węzła
                            y = new_element[m][1]
                            x_ind = int(np.around((x-x_coords[0])/element_x_length,0))  # liczymy indeksy węzła
                            y_ind = int(np.around((y-y_coords[0])/element_y_length,0))
                            
                            if Z[y_ind,x_ind] == -1:  # jeśli -1 to znaczy, że węzeł jeszcze nie był ponumerowany
                                list_of_nodes.append(new_element[m])  # numerujemy go
                                new_elementkeys.append(int(len(list_of_nodes) - 1))
                                Z[y_ind,x_ind] = int(len(list_of_nodes) - 1)
                            else:  # już był ponumerowany
                                new_elementkeys.append(int(Z[y_ind,x_ind]))  # sprawdzamy jaki ma numer i dodajemy go do tablicy     
                    else:  # trójkąt nieparzysty
                        new_element = [
                            [((primary_nodes[l, 0] + (j + 1) * element_x_length)), ((primary_nodes[l, 1] + k * element_y_length))],
                            [((primary_nodes[l, 0] + (j + 1) * element_x_length)),
                             ((primary_nodes[l, 1] + (k + 1) * element_y_length))],
                            [((primary_nodes[l, 0] + j * element_x_length)), ((primary_nodes[l, 1] + (k + 1) * element_y_length))]]
                        for m in range(3):  # numeracja nowych wierzchołków
                            x = new_element[m][0]
                            y = new_element[m][1]
                            x_ind = int(np.around((x-x_coords[0])/element_x_length,0))
                            y_ind = int(np.around((y-y_coords[0])/element_y_length,0))
                            if Z[y_ind,x_ind] == -1:  # numerujemy
                                list_of_nodes.append(new_element[m])  # numerujemy go
                                new_elementkeys.append(int(len(list_of_nodes) - 1))
                                Z[y_ind,x_ind] = int(len(list_of_nodes) - 1)
                            else:  # już był ponumerowany
                                new_elementkeys.append(int(Z[y_ind,x_ind]))  # sprawdzamy jaki ma numer i dodajemy go do tablicy    
    
                    # Uzupełniamy connection matrix
        
                    array_elements_keys[0, i] = new_elementkeys[0]
                    array_elements_keys[1, i] = new_elementkeys[1]
                    array_elements_keys[2, i] = new_elementkeys[2]

                    if debug_mode:
                        array_elements[i, 0, 0] = new_element[0][0]
                        array_elements[i, 0, 1] = new_element[0][1]
                        array_elements[i, 1, 0] = new_element[1][0]
                        array_elements[i, 1, 1] = new_element[1][1]
                        array_elements[i, 2, 0] = new_element[2][0]
                        array_elements[i, 2, 1] = new_element[2][1]

                    i += 1 # zwiększamy numer dla kolejnego elementu 
                    new_elementkeys = []  # czyścimy tablicę nowego elementu
                    if q == 1:
                        j += 1  # zwiększamy pierwszą współrzędną względem wierzchołka pierwotnego

            j = 0  # wracamy na start wiersza
            k += 1  # zwiększamy drugą współrzędną względem wierzchołka pierwotnego

    list_of_nodes = np.array(list_of_nodes) # z listy tworzymy numpy array

    if debug_mode:
        return array_elements_keys, list_of_nodes*scale_axes, array_elements, area_elements_list
    
    else:
        return array_elements_keys, list_of_nodes*scale_axes
